In [32]:
#导入模块
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms

In [33]:
#定义超参数
BATCH_SIZE=16
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS=10

In [34]:
#构架transforms
pipeline=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [35]:
#数据集
from torch.utils.data import DataLoader
#下载
train_set=datasets.MNIST("data",train=True,download=True,transform=pipeline)
test_set=datasets.MNIST("data",train=True,download=True,transform=pipeline)
#加载项
train_loader=DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [36]:
#网络模型
class Digit(nn.Module):
    def __init__(self):
            super().__init__()
            self.conv1=nn.Conv2d(1,10,5)
            self.conv2=nn.Conv2d(10,20,3)
            self.fc1=nn.Linear(20*10*10,500)
            self.fc2=nn.Linear(500,10)
            
    def forward(self,x):
        input_size=x.size(0)
        x=self.conv1(x)
        x=F.relu(x)
        x=F.max_pool2d(x, 2, 2)
        
        x=self.conv2(x)
        x=F.relu(x)
        
        x=x.view(input_size,-1)
        
        x=self.fc1(x)
        x=F.relu(x)
        
        x=self.fc2(x)
        
        output=F.log_softmax(x,dim=1)
        
        return output

In [37]:
# 定义优化器
model = Digit().to(DEVICE)

optimizer =optim.Adam(model.parameters(),lr=0.001)

In [38]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        pred = output.max(1, keepdim=True)
        loss.backward()
        optimizer.step()
        
        # 修改了字符串格式化的方式
        if batch_index % 3000 == 0:
            print(f"Train Epoch:{epoch}\t Loss:{loss.item():.6f}")

In [39]:
#定义测试方法
def test_model(model,device,test_loader):
    
    model.eval()
    
    correct=0.0
    
    test_loss=0.0
    with torch.no_qrad():
        for data,target in test_loader:
            data,target=data.to(device)
            output=model(data)
            test_loss+=f.coss_entropy(output,target().item())
            pred=output.max(1,keepdim=True)[1]
            
            correct+=pred.eq(target.view_as(pred)).sum().item()
        test_loss/=len(test_loader.dataset)
        print("test-Average loss:{:.4f},Accuracy:{:.3f}\n". format(test_loss,100.0*correct/len(test_loader.dataset)))

In [40]:
#调用方法
for epoch in range(1,EPOCHS+1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.281787
Train Epoch:1	 Loss:0.130298


AttributeError: module 'torch' has no attribute 'no_qrad'